# Жанровая классификация

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import keras
import matplotlib.pyplot as plt
import librosa
import tensorflow as tf

In [ ]:
tf.config.list_physical_devices()

## Фильтрация метаданных

In [ ]:
import glob


DATA_DIR = './data/fma_small/'
METADATA_DIR = './data/fma_metadata/'

mp3_files = glob.glob(DATA_DIR + '*/*.mp3')
mp3_names = list(map(lambda f: np.int64(f.split('/')[-1].split('.')[0]), mp3_files))

raw_tracks = pd.read_csv(METADATA_DIR + 'raw_tracks.csv')
tracks = raw_tracks[raw_tracks['track_id'].isin(mp3_names)]

tracks

## Сбор признаков, полученных с помощью `librosa`

In [ ]:
features_df = pd.read_csv(METADATA_DIR + 'features.csv', index_col=0, header=[0, 1, 2])
features_df = features_df[features_df.index.isin(mp3_names)]

features = np.unique(list(map(lambda x: x[0], list(features_df.columns))))

print(f"Features available: {features}")
print(f"Total: {len(features)}")

features_df

## Отбор признаков

Рассмотрим всю имеющуюся информацию о треках

In [ ]:
tracks.columns

Оценим число непустых значений тегов

In [ ]:
tracks['tags'].map(lambda x: None if x == '[]' else x).notnull().value_counts()

Подсчитаем число уникальных тегов

In [ ]:
from functools import reduce


unique_tags = reduce(lambda tags, l: tags.union(eval(l)), tracks['tags'], set())  
print(len(unique_tags))

Оставим предположительно полезную информацию из набора данных. Убедимся
в её необходимости позже.

In [ ]:
to_keep = [
  'track_id', "album_id", "artist_id", "track_duration", 
  "track_genres", "track_instrumental", "track_interest", "track_listens",
]

filtered_tracks = tracks[to_keep]
filtered_tracks

Преобразуем время в секунды

In [ ]:
def duration_to_int(t):
  splitted = t.split(":")
  
  return int(splitted[0]) * 60 + int(splitted[1])

filtered_tracks.loc[:,'track_duration'] = filtered_tracks.track_duration.apply(duration_to_int)
filtered_tracks

Узнаем количество жанров для треков

In [ ]:
import json


genres = filtered_tracks['track_genres'].map(lambda x: json.loads(x.replace("'", "\"")))
genre_ids = genres.map(lambda x: list(map(lambda y: y['genre_id'], x)))
genre_ids.map(lambda x: len(x)).value_counts()

Определим базовые жанры для каждого трека

In [ ]:
all_genres = pd.read_csv(METADATA_DIR + 'genres.csv')

base_genres = genre_ids.map(lambda x: all_genres[all_genres.genre_id == int(x[0])].iloc[0].top_level)

filtered_tracks['track_genres'] = base_genres
filtered_tracks

In [ ]:
base_genres.value_counts()

Получили 8 сбалансированных классов

In [ ]:
import seaborn as sns

def display_corr(df):
  corr = df.corr()
  cmap = sns.diverging_palette(230, 20, as_cmap=True)
  mask = np.triu(np.ones_like(corr, dtype=bool))
  sns.heatmap(corr, mask=mask, cmap=cmap)
  
display_corr(filtered_tracks)

Жанр трека очень плохо коррелирует с его длительностью, поэтому исключим
этот признак из рассмотрения

In [ ]:
filtered_tracks = filtered_tracks.drop('track_duration', axis=1)

Теперь добавим значения, предпосчитанные с помощью `librosa`

In [ ]:
merged = features_df.merge(filtered_tracks, how='inner', on='track_id')

display_corr(merged)

Конечно, признаков слишком много. Из всех возьмем признаки с наибольшей по
модулю корреляцией.

Для этого отсортируем признаки по степени корреляции

In [ ]:
correlation = merged.corr()

genres_corr = correlation['track_genres'].sort_values(key=lambda x: np.abs(x), ascending=False)
genres_corr

Изобразим распределение значений корреляции

In [ ]:
sns.histplot(genres_corr)

Видно, что наибольшее число признаков имеют почти нулевую корреляцию.
В связи с этим выберем наиболее информативные из них

In [ ]:
BOUNDARY = 0.2

selected = merged[genres_corr[abs(genres_corr) > BOUNDARY].reset_index()['index']]
selected

Перекодируем метки классов

In [ ]:
from sklearn.preprocessing import LabelEncoder


genre_le = LabelEncoder()

selected.track_genres = genre_le.fit_transform(selected.track_genres)
selected

In [ ]:
selected.columns = selected.columns.map(str)

In [ ]:
from sklearn.preprocessing import StandardScaler

for column in selected.columns:
  if column == 'track_genres':
    continue
  selected[column] = StandardScaler().fit_transform(selected[column].to_numpy().reshape(-1, 1))

Убедимся, что `StandardScaler` отработал корректно

In [ ]:
selected.describe()

## KNN

In [ ]:
from sklearn.model_selection import train_test_split


X = selected.drop('track_genres', axis=1)
y = selected['track_genres']

X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.33, random_state=69)

In [ ]:
def plot_score(n, scores):
    d = {'neighbors': n, 'score': scores}
    df = pd.DataFrame(d)

    sns.set(style='darkgrid')
    sns.lineplot(x='neighbors', y='score', data=df)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from tqdm import tqdm


ns = list(map(int, range(1, 100, 1)))
scores = []

for n in tqdm(ns):
    knn = KNeighborsClassifier(p=1, n_neighbors=n)
    knn.fit(X_train, y_train)
    Y_pred = knn.predict(X_test)
    scores.append(knn.score(X_test, y_test))

plot_score(ns, scores)

In [ ]:
from sklearn.model_selection import cross_val_score


scores = []

for n in tqdm(ns):
    knn = KNeighborsClassifier(p=1, n_neighbors=n)
    scores.append(cross_val_score(knn, X, y, cv=5).mean())

plot_score(ns, scores)